Importing Dependencies

In [17]:
import tensorflow as tf 
import tensorflow.keras
import numpy as np
import matplotlib.pyplot as plt
import functools
from IPython import display as ipythondisplay
from tqdm import tqdm
import time
import os
import sys
import h5py
import glob
import cv2
print(len(tf.config.list_physical_devices('GPU')) > 0)

True


Creating a data loader

In [18]:
class TrainLoader(object):
    def __init__(self, data_dir):
        print("Currently Opening")
        sys.stdout.flush()
        self.cache = h5py.File(data_dir, 'r')
        print("Loading")
        sys.stdout.flush()
        self.imgs = self.cache['images'][:]
        self.labels = self.cache['labels'][:].astype(np.float32)
        self.img_dim = self.imgs.shape
        sample_num = self.img_dim[0]
        self.train_idxs = np.random.permutation(np.arange(sample_num))
        self.positive_train_idxs = self.train_idxs[self.labels[self.train_idxs, 0] == 1.0]
        self.negative_train_idxs = self.train_idxs[self.labels[self.train_idxs, 0] != 1.0]

    def size(self):
        return self.train_idxs.shape[0]
    
    def step_per_epoch(self, batch_size):
        return self.size()//10//batch_size
    
    def gen_batch(self, n, p_positive=None, p_negative=None):
        positive_idxs = np.random.choice(self.positive_train_idxs, size=n//2, replace=False, p=p_positive)
        negative_idxs = np.random.choice(self.negative_train_idxs, size=n//2, replace=False, p=p_negative)
        idxs = np.concatenate((positive_idxs, negative_idxs))
        sorted_idxs = np.sort(idxs)
        img = (self.imgs[sorted_idxs,:,:,::-1]/255.).astype(np.float32)
        label = self.labels[sorted_idxs,...]
        return (img, label)
    
    def all_faces(self):
        return self.imgs[self.positive_train_idxs]

Gathering data from an online source. Found it while doing the deep learning specialization with stanford and deeplearning.ai

In [19]:
training_path = tensorflow.keras.utils.get_file('train_face.h5', 'https://www.dropbox.com/s/hlz8atheyozp1yx/train_face.h5?dl=1')
loader = TrainLoader(training_path)

Currently Opening
Loading


Creating a regular classifier

In [20]:
def classifier_creation(outputs=1):
    Conv = functools.partial(tensorflow.keras.layers.Conv2D, padding='same', activation='relu')
    BatchNorm = tensorflow.keras.layers.BatchNormalization
    Flatten = tensorflow.keras.layers.Flatten
    Dense = functools.partial(tensorflow.keras.layers.Dense, activation='relu')
    model = tensorflow.keras.Sequential([
        Conv(filters=1*12, kernel_size=5,  strides=2),
        BatchNorm(),
        Conv(filters=2*12, kernel_size=5,  strides=2),
        BatchNorm(),
        Conv(filters=4*12, kernel_size=3,  strides=2),
        BatchNorm(),
        Conv(filters=6*12, kernel_size=3,  strides=2),
        BatchNorm(),
        Flatten(),
        Dense(512),
        Dense(outputs, activation=None),
    ])
    return model

In [21]:
regular_classifier = classifier_creation()

Defining Hyperparameters

In [22]:
batch_size = 32
epochs = 2
lr = 1e-4

opt = tf.keras.optimizers.Adam(lr)

Training the Model

In [23]:
@tf.function
def train_step(imgs, labels):
    with tf.GradientTape() as tape:
        logits = regular_classifier(imgs)
        loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)

    #defining backprop
    gradients = tape.gradient(loss, regular_classifier.trainable_variables)
    opt.apply_gradients(zip(gradients, regular_classifier.trainable_variables))
    return loss
def training_loop():
    for epoch in range(epochs):
        for idx in tqdm(range(loader.size()//batch_size)):
            imgs, labels = loader.gen_batch(batch_size)
            loss = train_step(imgs, labels)

training_loop()

100%|██████████| 3434/3434 [00:23<00:00, 143.56it/s]


Getting the Accuracy of the normal classifier on the bias dataset

In [24]:
(b_x, b_y) = loader.gen_batch(1000)
y_pred = tf.round(tf.nn.sigmoid(regular_classifier.predict(b_x)))
reg_acc = tf.reduce_mean(tf.cast(tf.equal(b_y, y_pred), tf.float32))

print("Regular Classifier Accuracy: {:.4f}".format(reg_acc.numpy()))

Regular Classifier Accuracy: 0.9970
